In [4]:
from __future__ import print_function
import matplotlib.pyplot as plt
#%matplotlib notebook
#import matplotlib.pyplot as plt
import numpy as np
import numbers
import pywt
import random
import scipy
import skimage.color as color
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from skimage import data, img_as_float
from skimage.util import random_noise
from skimage.measure import compare_psnr
import os
from PIL import Image
import PIL
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.optim
from torch.autograd import Variable

from include import *

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print("num GPUs",torch.cuda.device_count())
#dtype = torch.FloatTensor

num GPUs 0


In [5]:
imgs = get_imgnet_imgs(num_samples = 10, path = '../imagenet/')

FileNotFoundError: [Errno 2] No such file or directory: '../imagenet/ILSVRC2012_val_00020180.JPEG'

In [4]:
def rep_error_deep_decoder(img_np,net,convert2ycbcr=False,upsample_mode='bilinear'):
    '''
    mse obtained by representing img_np with the deep decoder
    '''
    output_depth = img_np.shape[0]
    if output_depth == 3 and convert2ycbcr:
        img = rgb2ycbcr(img_np)
    else:
        img = img_np
    img_var = np_to_var(img).type(dtype)
    
    rnd = 500
    numit = 10000
    rn = 0.005
    mse_n, mse_t, ni, net = fit( num_channels=num_channels,
                        reg_noise_std=rn,
                        reg_noise_decayevery = rnd,
                        num_iter=numit,
                        LR=0.004,
                        img_noisy_var=img_var,
                        net=net,
                        img_clean_var=img_var,
                        find_best=True,
                        upsample_mode=upsample_mode,        
                        )
    out_img = net(ni.type(dtype)).data.cpu().numpy()[0]
    if output_depth == 3 and convert2ycbcr:
        out_img = ycbcr2rgb(out_img)
    return psnr(out_img,img_np), out_img, num_param(net)

output_depth=3

## Experiments that keep the number of parameters constant

In [5]:
def save_1d_list(li, filename):
    li = np.array(li)
    print("average and mean: ", np.mean(li),np.var(li))
    np.savetxt(filename, li.T, delimiter=' ')

In [6]:
psnrs = []
k = 16
num_channels = [k]*5

for img in imgs:
    stride=2
    padding = 1
    net = deconv_decoder(output_depth,num_channels_up=num_channels,filter_size=4,stride=stride,padding=padding).type(dtype)
    psnrv, out_img, nparam = rep_error_deep_decoder(img,net,upsample_mode="deconv")
    print(nparam)
    psnrs.append(psnrv)

save_1d_list(psnrs,"imagenet_fit_psnrs_deconv16.dat")

input shape:  [1, 16, 8, 8]
optimize with adam 0.004
24816tion 09990    Train loss 0.000209  Actual loss 0.000209 Actual loss orig 0.000209  Noise Energy 0.000000 
input shape:  [1, 16, 8, 8]
optimize with adam 0.004
24816tion 09990    Train loss 0.001795  Actual loss 0.001795 Actual loss orig 0.001795  Noise Energy 0.000000 
input shape:  [1, 16, 8, 8]
optimize with adam 0.004
24816tion 09990    Train loss 0.000161  Actual loss 0.000161 Actual loss orig 0.000161  Noise Energy 0.000000 
input shape:  [1, 16, 8, 8]
optimize with adam 0.004
24816tion 09990    Train loss 0.000894  Actual loss 0.000894 Actual loss orig 0.000894  Noise Energy 0.000000 
input shape:  [1, 16, 8, 8]
optimize with adam 0.004
24816tion 09990    Train loss 0.001250  Actual loss 0.001250 Actual loss orig 0.001250  Noise Energy 0.000000 
input shape:  [1, 16, 8, 8]
optimize with adam 0.004
24816tion 09990    Train loss 0.000197  Actual loss 0.000197 Actual loss orig 0.000197  Noise Energy 0.000000 
input shape:  [1

In [ ]:
psnrs4 = []
k = 64
num_channels = [k]*5

for img in imgs:
    net = fixed_decodernw(output_depth,num_channels_up=num_channels).type(dtype)  
    psnrv, out_img, nparamm = rep_error_deep_decoder(img,net,upsample_mode='none')
    psnrs4.append(psnrv)

save_1d_list(psnrs4,"imagenet_fit_psnrs_gaussian_64.dat")

In [9]:
psnrs5 = []
nparam = 25545
print(nparam)

for img in imgs:
    psnrv, out_img = rep_error_wavelet(img,ncoeff=int(nparam/output_depth))
    psnrs5.append(psnrv)

save_1d_list(psnrs5,"imagenet_fit_psnrs_wavelet_1_64.dat")

25545
average and mean:  31.867827675473922 24.267141197404392


In [6]:
psnrs = []
k = 64
num_channels = [k]*5

for img in imgs:
    net = decodernw(output_depth,num_channels_up=num_channels).type(dtype)
    psnrv, out_img, nparam = rep_error_deep_decoder(img,net)
    print(nparam)
    psnrs.append(psnrv)

save_1d_list(psnrs,"imagenet_fit_psnrs_up64.dat")

input shape:  [1, 64, 16, 16]
optimize with adam 0.004
25536tion 09990    Train loss 0.000484  Actual loss 0.000484 Actual loss orig 0.000484  Noise Energy 0.000000 
input shape:  [1, 64, 16, 16]
optimize with adam 0.004
25536tion 09990    Train loss 0.000416  Actual loss 0.000416 Actual loss orig 0.000416  Noise Energy 0.000000 
input shape:  [1, 64, 16, 16]
optimize with adam 0.004
25536tion 09990    Train loss 0.000166  Actual loss 0.000166 Actual loss orig 0.000166  Noise Energy 0.000000 
input shape:  [1, 64, 16, 16]
optimize with adam 0.004
25536tion 09990    Train loss 0.001618  Actual loss 0.001618 Actual loss orig 0.001618  Noise Energy 0.000000 
input shape:  [1, 64, 16, 16]
optimize with adam 0.004
25536tion 09990    Train loss 0.000101  Actual loss 0.000101 Actual loss orig 0.000101  Noise Energy 0.000000 
input shape:  [1, 64, 16, 16]
optimize with adam 0.004
25536tion 09990    Train loss 0.000715  Actual loss 0.000715 Actual loss orig 0.000715  Noise Energy 0.000000 
inpu